In [ ]:
import pandas as pd

In [ ]:
intent_list=['all','Solve_Professional_Problem','Factual_QA','Text_Assistant','Ask_for_Advice','Seek_Creativity','Leisure']
test_model_list=['deepseek-chat','gpt-3.5-turbo','glm-4','Baichuan2-Turbo','spark-3.5','qwen-max','gpt-4-0125-preview','ERNIE-Bot-4','moonshot-v1-8k','gpt-4','claude-3-opus-20240229','gpt-4-turbo','qwen2-72b-instruct']

field=intent_list[0]
test_model=test_model_list[-1]
df=pd.read_csv(f'../data/evaluation_field={field}_{test_model}_evaluation.csv')

In [ ]:
import re
# 首先匹配大括号内的所有内容

tag_dict={'Factual_QA':['Factuality','User Satisfaction','Clarity','Completeness','Logical Coherence','Overall'],\
                'Solve_Professional_Problem':['Factuality','User Satisfaction','Clarity','Logical Coherence','Completeness','Overall'],\
                'Text_Assistant':['Clarity','User Satisfaction','Logical Coherence','Factuality','Creativity','Overall'],\
                'Ask_for_Advice':['User Satisfaction','Factuality','Fairness and Responsibility','Creativity','Richness','Overall'],\
                'Seek_Creativity':['User Satisfaction','Logical Coherence','Creativity','Richness','Factuality','Overall'],\
                'Leisure':['User Satisfaction','Engagement','Appropriateness','Creativity','Factuality','Overall']}

# tag_list=tag_dict[field]

def extract_data(test_str,intent):
    flag=0
    tag_list=tag_dict[intent].copy()
    pattern_general = r'\{[^}]+\}'
    # 使用正则表达式搜索
    # general_match = re.search(pattern_general, test_str)
    general_match = re.findall(pattern_general, test_str)
    # 如果找到匹配项，进一步处理
    result_dict = {}
    ans_dict={}
    if general_match:
        matched_str=general_match[-1]
        final_ans_tag_list=['综合得分','Overall','总得分','Overall Score','总分','overall','Total Score','Final Score','总分数','综合评分']
        for ans_tag in final_ans_tag_list:
            tag_list[-1]=ans_tag

            pattern_fields = '|'.join([f'(?:\'|\"|’|‘)?{re.escape(tag)}(?:\'|\"|’|‘)?\s*:\s*(\d+)' for tag in tag_list])
    
            fields_matches = re.findall(pattern_fields, matched_str)
            for i, tag in enumerate(tag_list):
                for match in fields_matches:
                    if match[i]:
                        result_dict[tag] = int(match[i])
                        break
            if ans_tag in result_dict:
                ans_dict={}
                for key in result_dict:
                    if key!=ans_tag:
                        ans_dict[key]=result_dict[key]
                ans_dict['Overall']=result_dict[ans_tag]
                flag=1
                break
            result_dict = {}
    if flag==0:
        print("ERROR! type 2")
        print(result_dict)
        print(test_str)
    # 输出结果
    # print(result_dict)
    return ans_dict

In [ ]:
for index,row in df.iterrows():
    test_str=row['evaluation']
    if 'user_intent' in df.columns:
        intent=row['user_intent']
    else:
        intent=field
    ans_dict=extract_data(test_str,intent)
    # print(ans_dict)
    for key in ans_dict:
        df.loc[index,key]=ans_dict[key]

In [ ]:
df.to_csv(f'../data/finalScore_field={field}_LLM={test_model}.csv',encoding='utf-8-sig',index=False)